<a href="https://colab.research.google.com/github/tuphr2234/ChaosPredition/blob/main/ThreeBodySimulation_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Build Simulation

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import tensorflow as tf
import os
from pathlib import Path  


G=1


class Body:
  
  prevPosition=np.array([0,0])
  position=np.array([0,0])
  velocity=np.array([0,0])
  force=np.array([0,0])
  acceleration=np.array([0,0])
  mass=0

  def __init__(self,position,velocity,mass,name):
    self.position=np.array(position)
    self.velocity=np.array(velocity)
    #self.acceleration=np.array(acceleration)
    self.mass=mass
    self.prevPosition=np.array(position)
    self.name = name


  def setPosition(self,t):
    self.prevPosition = self.position
    self.position=self.position+self.velocity*t+(self.acceleration/2)* t**2
  
    


  def setVelocity(self,t):
    self.velocity=self.velocity+self.acceleration*t
  

  def getForce(self,body):
    softening = 0.00001
    Distance = body.prevPosition - self.position
    Rsquare = Distance[0]**2 + Distance[1]**2 + softening
    F = G* self.mass*body.mass / Rsquare  
    normalizeDis = Distance / math.sqrt(Rsquare)
    F = F*normalizeDis
    
    #print(f"body {self.name} force is {F} ")
    
    return F

  def setAcceleration(self,bodies):
    F=np.array([0,0])
    for body in bodies:
      F=F+self.getForce(body)
    self.acceleration = F/self.mass
   # print(f" {self.name} acceleration is {self.acceleration}")

In [2]:
def simulateOnTimeStep(bodies,t,steps,interval_in_data): # bodies: list of bodies. t: timestep . steps: number of steps. interval_in_data : frequency of rows taken to dataset: if equals 1 than it is every step
  dataset = []
  time = []

  for i in range(steps):
      list1=[]
      for body in bodies:
        list1.append(body.position)
      if i % interval_in_data ==0:
        dataset += list1
        time.append(i * t)
      
      for i,body in enumerate(bodies):
        body.setAcceleration([b for b in bodies if b is not body])
        body.setVelocity(t)
        body.setPosition(t)
       
        
      for body in bodies:
        #plt.plot([body.prevPosition[0],body.position[0]],[body.prevPosition[1],body.position[1]])
        body.prevPosition = body.position
  #plt.show()
    

  return np.array(dataset).reshape(steps//interval_in_data,len(bodies),2),time



In [3]:
def numpyToPandas(simulation,numOfBodies):
  columns = []

  for i in range(numOfBodies):
    columns.append(f'Body{i+1} x')
    columns.append(f'Body{i+1} y')
  simulation_reshaped = simulation.reshape(len(simulation), numOfBodies*2 )

  df = pd.DataFrame(simulation_reshaped, columns = columns)

  return df


Add Filters

In [4]:
#def addNoise(simulation):
#def removeBody(simulation)

Save Simulation

Prepare Dataset 

In [5]:
def prepareData(simulation,window_size_X):
  X = []
  y = []
  for i in range(len(simulation)-window_size_X):
    row = [bodies for bodies in simulation[i:i+window_size_X]]  ### adding time is an option
    X.append(row)
    label = simulation[i+window_size_X]
    y.append(label)
  return np.array(X) , np.array(y)  

In [6]:
def prepareData2(simulation,window_size_X,window_size_y):
  X = []
  y = []
  for i in range(len(simulation)-window_size_X):
    row = simulation[i:i+window_size_X]  ### adding time is an option
    X.append(row)
    label =  simulation[i+window_size_X:i+window_size_X + window_size_y]#######################################
    y.append(label)

  return np.array(X) , np.array(y)


In [7]:
def splitToTrainValidTest(dataset,testPercentage):
  trainLen = round((len(dataset)*(1-testPercentage-0.05)))
  valLen = round((len(dataset)*0.05))
  train_set = dataset[:trainLen]
  val_set = dataset[trainLen:trainLen+valLen]
  test_set = dataset[trainLen+valLen:]
  return train_set,val_set,test_set

In [8]:
def drawSimulation(simulation):
  prev = simulation[0]
  numOfBodies = len(simulation[0])
  for i,sim in enumerate(simulation):
    for j in range(0,len(sim),2):
      plt.plot([prev[j],sim[j]],[prev[j+1],sim[j+1]])
      prev[j] = sim[j]
      prev[j+1] = sim[j+1]
  plt.show()

In [ ]:
def showBodiesMovmentInGraph(simulation,trueForX):
 arr = []
 for i in range(0,len(simulation),2):
    for sim in simulation:
      arr.append(sim[i])


 plt.plot(arr)

Preprocessing/Postprocessing

Model

In [148]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
def buildModelLSTM(window_size_X,numberOfBodies):
  model = Sequential()
  model.add(InputLayer((window_size_X,numberOfBodies*2)))#model.add(InputLayer((window_size_X,numberOfBodies*2)))
  model.add(Conv1D(64, kernel_size=2))
  #model.add(Dropout(0.2))
  model.add(Conv1D(32, kernel_size=2))
  #model.add(TimeDistributed(conv_1d))
  model.add(Flatten())
  #model.add(LSTM(32))
  #model.add(Dropout(0.2))
  model.add(Dense(16, 'relu'))
  model.add(Dense(numberOfBodies*2, 'linear'))
  model.summary()
  return model



def buildModelLSTM2(window_size_X,window_size_y,numberOfBodies):
  model = Sequential()
  model.add(InputLayer((window_size_X,numberOfBodies*2)))#model.add(InputLayer((window_size_X,numberOfBodies*2)))
  model.add(Conv1D(64, kernel_size=2))
  #model.add(Dropout(0.2))
  model.add(Conv1D(32, kernel_size=2))
  #model.add(TimeDistributed(conv_1d))
  model.add(Flatten())
  #model.add(LSTM(32))
  #model.add(Dropout(0.2))
  model.add(Dense(16, 'relu'))
  model.add(Dense(numberOfBodies*2*window_size_y, 'linear'))
  model.add(tf.keras.layers.Reshape((window_size_y, numberOfBodies*2)))
  model.summary()
  return model

In [10]:
def preprocessSimulation(simulation):
  shape = simulation[0].shape
  newSimulation = np.append(simulation[1:],[np.zeros(shape)],axis=0) - simulation
  newSimulation = newSimulation[:-1]
  #for i,s in enumerate(newSimulation):
  #  s = s / np.linalg.norm(simulation[i])
  return newSimulation

In [11]:
def postprocess(lastKnownElement , listOfPredictedChanges):
  listOfPredictions = []
  for pred in listOfPredictedChanges:
    #newElement = (pred * np.linalg.norm(lastKnownElement)) + lastKnownElement
    newElement = pred  + lastKnownElement
    listOfPredictions.append(newElement)
    lastKnownElement = newElement
  return np.array(listOfPredictions)  


Training Model

In [48]:
#position,velocity,mass
#%%timeit
bodies=[]
bodies.append(Body([-0.97000436, 0.24308753],[0.4662036850, 0.4323657300],1,"body1"))
bodies.append(Body([0,0],[-0.93240737, -0.86473146],1,"body2"))
bodies.append(Body([0.97000436, -0.24308753],[0.4662036850, 0.4323657300],1,"body3"))
#bodies.append(Body([2, -0.24308753],[-0.4662036850, 0.4323657300],1,"body4"))

 #r1(0) = −r3(0) = (−0.97000436, 0.24308753);
 # r2(0) = (0,0); v1(0) = v3(0) = (0.4662036850, 0.4323657300);
 # v2(0) = (−0.93240737, −0.86473146). The values are obtained from Chenciner & Montgomery (2000).







numOfBodies=len(bodies)

##t: timestep . steps: number of steps. interval_in_data
timestep = 0.001 # actual seconds
steps = 122000
interval_in_data = 10

simulation1,timesteps = simulateOnTimeStep(bodies,timestep,steps,interval_in_data)
simulation1=simulation1.reshape(len(simulation1), numOfBodies*2 )


In [ ]:
drawSimulation(simulation1)

bodies2=[]
bodies2.append(Body([-1, 0],[0.6976850, 0.4323657300],1,"body1"))
bodies2.append(Body([0,1],[0.23540737, -0.86473146],1.1,"body2"))
bodies2.append(Body([1, 0],[-0.4662036850, -0.1823657300],1,"body3"))
#bodies.append(Body([2, -0.2308753],[0.4662036850, 0.4323657300],1,"body4"))

numOfBodies2=len(bodies2)

##t: timestep . steps: number of steps. interval_in_data
timestep = 0.01 # actual seconds
steps = 1000000
interval_in_data = 100

simulation2,timesteps = simulateOnTimeStep(bodies2,timestep,steps,interval_in_data)
simulation2=simulation2.reshape(len(simulation2), numOfBodies2*2 )

In [50]:
## save simulation as csv
def saveSimulation(simulation,numOfBodies,simulation_name):
  df = numpyToPandas(simulation,numOfBodies) 
  df.to_csv(f'/content/{simulation_name}.csv')

In [51]:
#saveSimulation(simulation1,numOfBodies,"best of chaos")

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.signal import hilbert
from scipy.fft import fft, ifft
scaled = False 
windowSizeX= 30
windowSizeY = 10
#batch = 6

derivativeSimulation1 = preprocessSimulation(simulation1)

train, val, test = splitToTrainValidTest(derivativeSimulation1,0.2)

if scaled:
  scaler = StandardScaler()
  scaler.fit(simulation1[:len(train)+len(val)])
  #valScaler.fit(train)  ### train / val as input ? 

  train = scaler.transform(train)
  val = scaler.transform(val)
  #train = hilbert(train)
  #val = hilbert(val)
#y_train = scaler.fit(train)

#train = preprocessSimulation(train)
#val = preprocessSimulation(val)



#train2, val2, test2 = splitToTrainValidTest(simulation2,0.2)


X_train, y_train = prepareData(train,windowSizeX)
X_val, y_val = prepareData(val,windowSizeX)
X_test, y_test = prepareData(test,windowSizeX)




model =  buildModelLSTM(windowSizeX,numOfBodies)
cp1 = ModelCheckpoint('model/', save_best_only=True)
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])


#X_train2, y_train2 = prepareData(train2,windowSizeX)
#X_val2, y_val2 = prepareData(val2,windowSizeX)
#X_tes2t, y_test2 = prepareData(test2,windowSizeX)

#generator = TimeseriesGenerator(train_scaled,train_scaled,length = len(test),batch_size=batch)

In [ ]:
#for i in range(100):
  #model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=2, callbacks=[cp1])
  #model.fit(X_train2, y_train2, validation_data=(X_val2, y_val2), epochs=2, callbacks=[cp1])
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, callbacks=[cp1])

In [ ]:
#history = model.fit_generator(generator,epochs=100,verbose=1)

Predictions

In [160]:
from sklearn.metrics import mean_squared_error

In [175]:
predictions = model.predict(X_test)          ###############  LOCAL RMSE  (prediction per window)
#predictions = postprocess(val[-1],predictions)
#predictions = scaler.inverse_transform(predictions)
print(math.sqrt(mean_squared_error(test[windowSizeX:],predictions)))


0.002013177611595624


In [152]:
def slidingWindowActualPredictions(model,windowArr,windowSize,NumOfBodies,predictionsRequestedNumber):
  

  actualPredictionArr =[]
  for i in range(predictionsRequestedNumber):

    prediction = model.predict(windowArr[-1:]) ### predict from last window
    actualPredictionArr.append(prediction)
    temp = np.zeros([windowSize,numOfBodies*2])
    temp = temp + windowArr[-1]
    temp =temp[1:]
    newWindow = np.append(temp.reshape(windowSize-1,numOfBodies*2),prediction)
    newWindow = newWindow.reshape(windowSize,numOfBodies*2)
    lengthX = len(windowArr)
    windowArr = np.append(windowArr,newWindow) ### add new window that the last vector is the prediction 
    windowArr = windowArr.reshape(lengthX+1,windowSize,numOfBodies*2)

  return actualPredictionArr

In [153]:
import math

listOfPredictions = slidingWindowActualPredictions(model,X_train,windowSizeX,numOfBodies,len(val)+len(test))
listOfPredictions =np.array(listOfPredictions)
listOfPredictions = listOfPredictions.squeeze()

listOfPredictionsTrue = postprocess(simulation1[len(X_train)+windowSizeX-1],listOfPredictions)

if scaled:
  listOfPredictionsTrue = scaler.inverse_transform(listOfPredictionsTrue)


In [155]:
listOfPredictions[1000]

array([nan, nan, nan, nan, nan, nan], dtype=float32)

In [ ]:
#listOfPredictionsTrue = postprocess(simulation1[len(X_train)+windowSizeX-1],listOfPredictions)


In [176]:
print(math.sqrt(mean_squared_error(simulation1[len(X_train)+windowSizeX+1:],listOfPredictionsTrue))) ##### ACTUAL RMSE #####


ValueError: ignored

In [ ]:
print((simulation1[len(X_train)+windowSizeX]))

print((train[-1:]))
#y_train =sc.inverse_transform(y_train)
#print((y_train[-1:]))
print(listOfPredictions[0])
if scaled:
  print((scaler.inverse_transform(model.predict(X_train[-1:]))))

In [ ]:
import matplotlib.pyplot as plt
disArr = []
for i in range(len(val)+len(test)):
  disArr.append(np.linalg.norm(simulation1[len(X_train)+windowSizeX+i]-listOfPredictionsTrue[i]))

plt.plot(disArr)  

In [ ]:
drawSimulation(simulation1[len(X_train)+windowSizeX:len(X_train)+windowSizeX+60])

In [ ]:
drawSimulation(listOfPredictionsTrue[:60])

In [ ]:
simulation1.shape

In [ ]:
bodies=[]
bodies.append(Body([-1, 0],[0.4662036860, 0.4323657300],1,"body1"))
bodies.append(Body([0,0],[-0.93240737, -0.86473146],1.1,"body2"))
bodies.append(Body([1, 0],[0.4662036850, 0.4323657300],1,"body3"))
bodies.append(Body([2, -0.24308753],[-0.4662036850, -0.4323657300],1,"body4"))


simulation2 = simulateOnTimeStep(bodies,0.01,100000,10)

In [ ]:
distance = simulation1 - simulation2
curentDis=0
disArr = []
for i in range(100000):
  for j in range(4):
    curentDis = curentDis + math.sqrt(distance[i][j][0]**2 + distance[i][j][1]**2) 
  disArr.append(curentDis)
  curentDis=0

In [ ]:
time=[]
for i in range(100000):
  time.append(i*0.01)
plt.plot(time[0:10000],disArr[0:10000])
plt.show()

In [ ]:
disArr = np.array(disArr)
shiftdisArr=disArr[1:]
shiftdisArr=np.append(shiftdisArr,[0])
len(shiftdisArr)


In [ ]:
disArrderiv = (shiftdisArr - disArr)/0.01

In [ ]:
time=[]
for i in range(100000):
  time.append(i*0.01)
plt.plot(time[500:40000],disArrderiv[500:40000])
plt.show()

test